### Preliminary data cleaning:
- Combine datasets into one 
- Clean data names 
- Make Aboard into dictionaries 
- Replace "?" with nan 
- add Decades column 
- Fix date time column
- Define data types 
- Assumptions "?" == 0

In [1]:
import pandas as pd 
import numpy as np
import datetime as dt 
from functions.extraction_and_cleaning import read_bronze

In [2]:
df = read_bronze()
df.head(5)

,Date,Time,Location,Operator,Flight #,Route,AC\n Type,Registration,cn / ln,Aboard,Fatalities,Ground,Summary
0,"January 05, 1953",2139,"Belfast, Northern Ireland",British European Airways,?,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,35 (passengers:31 crew:4),27 (passengers:24 crew:3),0,Crashed into approach lights while attempting ...
1,"January 07, 1953",2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,7 (passengers:3 crew:4),7 (passengers:3 crew:4),0,The plane struck a tree on a mountain ridge at...
2,"January 07, 1953",0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,40 (passengers:37 crew:3),40 (passengers:37 crew:3),0,The plane crashed into mountains while en rout...
3,"January 15, 1953",0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,?,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,?,26 (passengers:16 crew:10),26 (passengers:16 crew:10),0,Both aircraft crashed after a midair collision...
4,"January 26, 1953",1144,"Sinnai, Sardinia, Italy",Linee Aeree Italiane,?,Cagliari - Rome,Douglas DC-3,I-LAIL,4308,19 (passengers:15 crew:4),19 (passengers:15 crew:4),0,Crashed 10 miles east of Cagliari in the Sinna...


First lets fix the column names

In [3]:
new_names = []
for item in df.columns:
    item = item.lower()
    item = item.strip()
    item = item.replace("\n"," ")
    item = item.replace("/"," ")
    item = item.replace("#","num")
    item = item.split()
    item = "_".join(item)
    new_names.append(item.title())

In [4]:
df.columns = new_names

In [5]:
def clean_aboard(x):
    x = x.replace("(","").replace(")","").split()
    a, b, c = x[0], x[1].split(":")[-1], x[2].split(":")[-1]
    try: 
        return {"total":int(a),"passengers":int(b),"crew":int(c)}
    except:
        if a == "?":
            a = float("nan")
        else:
            a = int(a)
        if b == "?":
            b = float("nan")
        else: 
            b = int(b)
        if c == "?":
            c = float("nan")
        else: 
            c = int(c)
    return {"total":a,"passengers":b,"crew":c}

In [6]:
df.Aboard = df.Aboard.apply(clean_aboard)
df.Fatalities = df.Fatalities.apply(clean_aboard)
df.head(3)

,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary
0,"January 05, 1953",2139,"Belfast, Northern Ireland",British European Airways,?,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0,Crashed into approach lights while attempting ...
1,"January 07, 1953",2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0,The plane struck a tree on a mountain ridge at...
2,"January 07, 1953",0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0,The plane crashed into mountains while en rout...


In [7]:
# Example getting flights with total passengers  + crew greater than 500 
df[df.Aboard.apply(lambda x: x["total"] > 500)]

,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary
293,"August 12, 1985",1856,"Mt. Osutaka, near Ueno Village, Japan",Japan Air Lines,123,Tokyo - Osaka,Boeing B-747-SR46,JA8119,20783/230,"{'total': 524, 'passengers': 509, 'crew': 15}","{'total': 520, 'passengers': 505, 'crew': 15}",0,The aircraft suffered an aft pressure bulkhead...
3640,"July 23, 1999",1125,"Tokyo, Japan",All Nippon Airways,61,"Tokyo, Japan - Chitose, Japan",Boeing B-747,JA8966,27442/1066,"{'total': 517, 'passengers': 503, 'crew': 14}","{'total': 1, 'passengers': 0, 'crew': 1}",0,Two minutes after taking off from Haneda Airpo...
4639,"March 27, 1977",1707,"Tenerife, Canary Islands",Pan American World Airways / KLM,1736/4805,Tenerife - Las Palmas / Tenerife - Las Palmas,Boeing B-747-121 / Boeing B-747-206B,N736PA/PH-BUF,19643/11 / 20400/157,"{'total': 644, 'passengers': 614, 'crew': 30}","{'total': 583, 'passengers': 560, 'crew': 23}",0,Both aircraft were diverted to Tenerife becaus...


In [8]:
# df.Ground = df.Ground.astype("Int64") 

In [9]:
df[df == "?"] = float("nan")
df["Date_2"] = pd.to_datetime(df.Date)
# df["Date_2"]= df.Date.apply(lambda x: x.dt.strftime('% B % d, % Y,'))
df.head(3)

,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Date_2
0,"January 05, 1953",2139,"Belfast, Northern Ireland",British European Airways,NaN,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0,Crashed into approach lights while attempting ...,1953-01-05
1,"January 07, 1953",2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0,The plane struck a tree on a mountain ridge at...,1953-01-07
2,"January 07, 1953",0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0,The plane crashed into mountains while en rout...,1953-01-07


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5037 entries, 0 to 5036
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          5037 non-null   object        
 1   Time          3529 non-null   object        
 2   Location      5032 non-null   object        
 3   Operator      5026 non-null   object        
 4   Flight_Num    1341 non-null   object        
 5   Route         4273 non-null   object        
 6   Ac_Type       5022 non-null   object        
 7   Registration  4761 non-null   object        
 8   Cn_Ln         4366 non-null   object        
 9   Aboard        5037 non-null   object        
 10  Fatalities    5037 non-null   object        
 11  Ground        4993 non-null   object        
 12  Summary       4978 non-null   object        
 13  Date_2        5037 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(13)
memory usage: 551.1+ KB


In [11]:
df["Decade"] = df.Date_2.dt.year // 10 * 10 

In [12]:
df = df[['Decade', 'Date','Time', 'Location', 'Operator', 'Flight_Num', 'Route',
       'Ac_Type', 'Registration', 'Cn_Ln', 'Aboard', 'Fatalities', 'Ground',
       'Summary']]

In [13]:
df.head(5)

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary
0,1950,"January 05, 1953",2139,"Belfast, Northern Ireland",British European Airways,NaN,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0,Crashed into approach lights while attempting ...
1,1950,"January 07, 1953",2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0,The plane struck a tree on a mountain ridge at...
2,1950,"January 07, 1953",0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0,The plane crashed into mountains while en rout...
3,1950,"January 15, 1953",0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,NaN,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,NaN,"{'total': 26, 'passengers': 16, 'crew': 10}","{'total': 26, 'passengers': 16, 'crew': 10}",0,Both aircraft crashed after a midair collision...
4,1950,"January 26, 1953",1144,"Sinnai, Sardinia, Italy",Linee Aeree Italiane,NaN,Cagliari - Rome,Douglas DC-3,I-LAIL,4308,"{'total': 19, 'passengers': 15, 'crew': 4}","{'total': 19, 'passengers': 15, 'crew': 4}",0,Crashed 10 miles east of Cagliari in the Sinna...


In [14]:
df[df.Aboard.apply(lambda x: isinstance(x['total'],float))]

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary
20,1950,"April 10, 1953",NaN,"Near Anchieta Island, Brazil",REAL,NaN,NaN,Doublas Dc-3,NaN,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': 26, 'passengers': nan, 'crew': nan}",NaN,NaN
400,1940,"March 18, 1946",NaN,"Between Chungking and Shanghai, China",China National Aviation Corporation,NaN,Chunking - Shanghai,NaN,139,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': nan, 'passengers': nan, 'crew': nan}",NaN,Disappeared while en route. Plane never located.
535,1950,"December 22, 1952",NaN,"Damascus, Syria",Syrian Air,NaN,NaN,Douglas DC-3,NaN,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': 9, 'passengers': nan, 'crew': nan}",NaN,NaN
635,1940,"November 09, 1944",NaN,"Seljord, Norway",Military - U.S. Army Air Corps,NaN,NaN,NaN,42-52196,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': nan, 'passengers': nan, 'crew': nan}",NaN,NaN
833,1990,"February 20, 1992",NaN,En route from Argentina to California,Aerolineas Argentinas,386,NaN,Boeing B-747,NaN,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': 1, 'passengers': nan, 'crew': nan}",0,A passenger died of cholera while en route.
950,1950,"January 20, 1951",NaN,"Huilyo, Peru",Military - TAM,NaN,NaN,Douglas DC-3,NaN,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': 16, 'passengers': nan, 'crew': nan}",NaN,NaN
1616,1950,"March 13, 1954",NaN,"Dien Bien Phu, South Vietnam",Aigle Azur,NaN,NaN,C-46,NaN,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': nan, 'passengers': nan, 'crew': nan}",NaN,Shot down.
1710,1950,"June 13, 1956",NaN,Over North Carolina,Piedmont,NaN,NaN,Douglas DC-3,NaN,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': 1, 'passengers': 1, 'crew': nan}",0,A passenger fell to his death after the door o...
1765,1940,"October 01, 1942",NaN,"Kunming, China",China National Aviation Corporation,NaN,NaN,Douglas C-47,69,NaN,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': nan, 'passengers': nan, 'crew': nan}",NaN,Crashed while attempting to land after losing ...
2073,1950,"April 20, 1957",NaN,"Jirkouk, Iraq",Air France,NaN,"Tehran, Iran - Istanbul, Turkey",Lockheed Super Constellation,F-BGNE,4514,"{'total': nan, 'passengers': nan, 'crew': nan}","{'total': 1, 'passengers': 1, 'crew': 0}",0,Explosive decompression. A passenger was sucke...


In [15]:
# def fix_time_avoid_na(x):
#     if x == np.nan:
#         return x 
#     else:
#          x = str(x)
#          return x[:2] + ":" + x[2:]


In [16]:
df.to_json('data/silver/1920_2024.json')